# Sequence Analysis with Python

Contact: Veli Mäkinen veli.makinen@helsinki.fi

The following assignments introduce applications of hashing with ```dict()``` primitive of Python. While doing so, a rudimentary introduction to biological sequences is given. 
This framework is then enhanced with probabilities, leading to routines to generate random sequences under some constraints, including a general concept of *Markov-chains*. All these components illustrate the usage of ```dict()```, but at the same time introduce some other computational routines to efficiently deal with probabilities.   
The function ```collections.defaultdict``` can be useful.

Below are some "suggested" imports. Feel free to use and modify these, or not. Generally it's good practice to keep most or all imports in one place. Typically very close to the start of notebooks.

In [2]:
from collections import defaultdict
from itertools import product

import numpy as np
from numpy.random import choice

The automated TMC tests do not test cell outputs. These are intended to be evaluated in the peer reviews. So it is still be a good idea to make the outputs as clear and informative as possible.

To keep TMC tests running as well as possible it is recommended to keep global variable assignments in the notebook to a minimum to avoid potential name clashes and confusion. Additionally you should keep all actual code exection in main guards to keep the test running smoothly. If you run [check_sequence.py](https://raw.githubusercontent.com/saskeli/data-analysis-with-python-summer-2019/master/check_outputs.py) in the `part07-e01_sequence_analysis` folder, the script should finish very quickly and optimally produce no output.

If you download data from the internet during execution (codon usage table), the parts where downloading is done should not work if you decide to submit to the tmc server. Local tests should work fine.

## DNA and RNA

A DNA molecule consist, in principle, of a chain of smaller molecules. These smaller molecules have some common basic components (bases) that repeat. For our purposes it is sufficient to know that these bases are nucleotides adenine, cytosine, guanine, and thymine with abbreviations ```A```, ```C```, ```G```, and ```T```. Given a *DNA sequence* e.g. ```ACGATGAGGCTCAT```, one can reverse engineer (with negligible loss of information) the corresponding DNA molecule.

Parts of a DNA molecule can *transcribe* into an RNA molecule. In this process, thymine gets replaced by uracil (```U```). 


1. Write a function ```dna_to_rna``` to convert a given DNA sequence $s$ into an RNA sequence. For the sake of exercise, use ```dict()``` to store the symbol to symbol encoding rules. Create a program to test your function.

In [3]:

def dna_to_rna(s):
    rule = dict(zip("ACGT", "ACGU"))
    return "".join(rule[x] for x in s)
    
if __name__ == '__main__':
    print(dna_to_rna("AACGTGATTTC"))

AACGUGAUUUC


### Idea of solution

The idea of solution is to replace each character in the string s with the value of the stored dictionary key for the character. 

### Discussion

The results were expected and the program worked well.

## Proteins

Like DNA and RNA, protein molecule can be interpreted as a chain of smaller molecules, where the bases are now amino acids. RNA molecule may *translate* into a protein molecule, but instead of base by base, three bases of RNA correspond to one base of protein. That is, RNA sequence is read triplet (called codon) at a time. 

2. Consider the codon to amino acid conversion table in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html. Write a function ```get_dict``` to read the table into a ```dict()```, such that for each RNA sequence of length 3, say $\texttt{AGU}$, the hash table stores the conversion rule to the corresponding amino acid. You may store the html page to your local src directory,
and parse that file.

In [4]:

def get_dict():
    data = []
    with open("Cut.txt", "r") as file:
        for line in file:
            data.extend(line.strip().split(")  "))
    dct = {}
    for codon in data:
        parts = codon.split(" ")
        dct[parts[0]] = parts[1]

    return dct
    
if __name__ == '__main__':
    codon_to_aa = get_dict()
    print(codon_to_aa)

{'UUU': 'F', 'UCU': 'S', 'UAU': 'Y', 'UGU': 'C', 'UUC': 'F', 'UCC': 'S', 'UAC': 'Y', 'UGC': 'C', 'UUA': 'L', 'UCA': 'S', 'UAA': '*', 'UGA': '*', 'UUG': 'L', 'UCG': 'S', 'UAG': '*', 'UGG': 'W', 'CUU': 'L', 'CCU': 'P', 'CAU': 'H', 'CGU': 'R', 'CUC': 'L', 'CCC': 'P', 'CAC': 'H', 'CGC': 'R', 'CUA': 'L', 'CCA': 'P', 'CAA': 'Q', 'CGA': 'R', 'CUG': 'L', 'CCG': 'P', 'CAG': 'Q', 'CGG': 'R', 'AUU': 'I', 'ACU': 'T', 'AAU': 'N', 'AGU': 'S', 'AUC': 'I', 'ACC': 'T', 'AAC': 'N', 'AGC': 'S', 'AUA': 'I', 'ACA': 'T', 'AAA': 'K', 'AGA': 'R', 'AUG': 'M', 'ACG': 'T', 'AAG': 'K', 'AGG': 'R', 'GUU': 'V', 'GCU': 'A', 'GAU': 'D', 'GGU': 'G', 'GUC': 'V', 'GCC': 'A', 'GAC': 'D', 'GGC': 'G', 'GUA': 'V', 'GCA': 'A', 'GAA': 'E', 'GGA': 'G', 'GUG': 'V', 'GCG': 'A', 'GAG': 'E', 'GGG': 'G'}


### Idea of solution

The idea of solution is to first store the conversion table in the html to local src directory (file Cut.txt) and parse that file into a list of lines each containing a single sequence information. Second form a dictionary from the list with sequence as key and corresponding amino acid as value. 

### Discussion

Other than storing the conversion table from the html, the program works well and results are as expected. Html table was parsed into Cut.txt file by hand.

3. Use the same conversion table as above, but now write function `get_dict_list` to read the table into a `dict()`, such that for each amino acid the hash table stores the list of codons encoding it.    

In [5]:

def get_dict_list():
    data = []
    with open("Cut.txt", "r") as file:
        for line in file:
            data.extend(line.strip().split(")  "))
    dct = {}
    for codon in data:
        parts = codon.split(" ")
        triplet, amino = parts[0], parts[1]
        if amino not in dct:
            dct[amino] = []
        dct[amino].append(triplet)

    return dct
    
if __name__ == '__main__':
    aa_to_codons = get_dict_list()
    print(aa_to_codons)

{'F': ['UUU', 'UUC'], 'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'], 'Y': ['UAU', 'UAC'], 'C': ['UGU', 'UGC'], 'L': ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'], '*': ['UAA', 'UGA', 'UAG'], 'W': ['UGG'], 'P': ['CCU', 'CCC', 'CCA', 'CCG'], 'H': ['CAU', 'CAC'], 'R': ['CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 'Q': ['CAA', 'CAG'], 'I': ['AUU', 'AUC', 'AUA'], 'T': ['ACU', 'ACC', 'ACA', 'ACG'], 'N': ['AAU', 'AAC'], 'K': ['AAA', 'AAG'], 'M': ['AUG'], 'V': ['GUU', 'GUC', 'GUA', 'GUG'], 'A': ['GCU', 'GCC', 'GCA', 'GCG'], 'D': ['GAU', 'GAC'], 'G': ['GGU', 'GGC', 'GGA', 'GGG'], 'E': ['GAA', 'GAG']}


### Idea of solution

The idea of solution is similar to the previous solution, but the dictionary is formed with amino acid as key and a list of corresponding RNA sequences as value. 

### Discussion

The program works well and results are as expected.

With the conversion tables at hand, the following should be trivial to solve.

4. Fill in function ```rna_to_prot``` in the stub solution to convert a given DNA sequence $s$ into a protein sequence. 
You may use the dictionaries from exercises 2 and 3. You can test your program with `ATGATATCATCGACGATGTAG`.

In [6]:
import re

def rna_to_prot(s):
    codons = re.findall('...', s)
    dct = get_dict()
    return "".join(dct[x] for x in codons)

def dna_to_prot(s):
    return rna_to_prot(dna_to_rna(s))

if __name__ == '__main__':
    print(dna_to_prot("ATGATATCATCGACGATGTAG"))

MISSTM*


### Idea of solution

Idea of solution is to split the string s into RNA sequences with length of three and save them into a list, then form the conversion rule dictionary with function made in the first solution, and eventually form the DNA sequence by going using the values in the list as keys to the conversion rule dictionary.

### Discussion

The program works well and results are as expected.

You may notice that there are $4^3=64$ different codons, but only 20 amino acids. That is, some triplets encode the same amino acid.  

## Reverse translation

It has been observed that among the codons coding the same amino acid, some are more frequent than others. These frequencies can be converted to probabilities. E.g. consider codons `AUU`, `AUC`, and `AUA` that code for amino acid isoleucine.
If they are observed, say, 36, 47, 17 times, respectively, to code isoleucine in a dataset, the probability that a random such event is `AUU` $\to$ isoleucine is 36/100.

This phenomenon is called *codon adaptation*, and for our purposes it works as a good introduction to generation of random sequences under constraints.   

5. Consider the codon adaptation frequencies in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html and read them into a ```dict()```, such that for each RNA sequence of length 3, say `AGU`, the hash table stores the probability of that codon among codons encoding the same amino acid.
Put your solution in the ```get_probabability_dict``` function. Use the column "([number])" to estimate the probabilities, as the two preceding columns contain truncated values.  

In [7]:

def get_probabability_dict():
    data = []
    with open("Cut.txt", "r") as file:
        for line in file:
            data.extend(line.strip().split(")  "))
    dct = {}
    for codon in data:
        parts = codon.split(" ")
        triplet, prob = parts[0], parts[2]
        dct[triplet] = float(prob)

    return dct
    
if __name__ == '__main__':
    codon_to_prob = get_probabability_dict()
    items = sorted(codon_to_prob.items(), key=lambda x: x[0])
    for i in range(1 + len(items)//6):
        print("\t".join(
            f"{k}: {v:.6f}"
            for k, v in items[i*6:6+i*6]
        ))

AAA: 0.430000	AAC: 0.530000	AAG: 0.570000	AAU: 0.470000	ACA: 0.280000	ACC: 0.360000
ACG: 0.110000	ACU: 0.250000	AGA: 0.210000	AGC: 0.240000	AGG: 0.210000	AGU: 0.150000
AUA: 0.170000	AUC: 0.470000	AUG: 1.000000	AUU: 0.360000	CAA: 0.270000	CAC: 0.580000
CAG: 0.730000	CAU: 0.420000	CCA: 0.280000	CCC: 0.320000	CCG: 0.110000	CCU: 0.290000
CGA: 0.110000	CGC: 0.180000	CGG: 0.200000	CGU: 0.080000	CUA: 0.070000	CUC: 0.200000
CUG: 0.400000	CUU: 0.130000	GAA: 0.420000	GAC: 0.540000	GAG: 0.580000	GAU: 0.460000
GCA: 0.230000	GCC: 0.400000	GCG: 0.110000	GCU: 0.270000	GGA: 0.250000	GGC: 0.340000
GGG: 0.250000	GGU: 0.160000	GUA: 0.120000	GUC: 0.240000	GUG: 0.460000	GUU: 0.180000
UAA: 0.300000	UAC: 0.560000	UAG: 0.240000	UAU: 0.440000	UCA: 0.150000	UCC: 0.220000
UCG: 0.050000	UCU: 0.190000	UGA: 0.470000	UGC: 0.540000	UGG: 1.000000	UGU: 0.460000
UUA: 0.080000	UUC: 0.540000	UUG: 0.130000	UUU: 0.460000


### Idea of solution

The idea of the solution is similar to the second solution, but in this case the value corresponding to a key in the dictionary is the probability of the codon encoding the same amino acid. 

### Discussion

The program works well and results are as expected.

Now you should have everything in place to easily solve the following.


6. Write a class ```ProteinToMaxRNA``` with a ```convert``` method which converts a protein sequence into the most likely RNA sequence to be the source of this protein. Run your program with `LTPIQNRA`.

In [8]:
class ProteinToMaxRNA:
    
    def __init__(self):
        self.aminos = get_dict_list()
        self.codon_prob = get_probabability_dict()

    def convert(self, s):
        protein = ""
        for c in s:
            codons = self.aminos[c]
            m = 0
            for x in codons:
                if self.codon_prob[x] > m:
                    m = self.codon_prob[x]
                    cod = x
            protein = protein + cod
        return protein


if __name__ == '__main__':
    protein_to_rna = ProteinToMaxRNA()
    print(protein_to_rna.convert("LTPIQNRA"))

CUGACCCCCAUCCAGAACAGAGCC


### Idea of solution

The class is initiated with two dictionaries returned from functions in earlier solutions: aminos-dictionary is from solution three and codon_prob from solution five. The protein is gradually build in the convert-function by going through the list corresponding to the encoding codons of the amino acid in question. The chosen codon is the one with highest probability codon for the amino acid.   

### Discussion

The program works well and results are as expected.

Now we are almost ready to produce random RNA sequences that code a given protein sequence. For this, we need a subroutine to *sample from a probability distribution*. Consider our earlier example of probabilities 36/100, 47/100, and 17/100 for `AUU`, `AUC`, and `AUA`, respectively. 
Let us assume we have a random number generator ```random()``` that returns a random number from interval $[0,1)$. We may then partition the unit interval according to cumulative probabilities to $[0,36/100), [36/100,83/100), [83/100,1)$, respectively. Depending which interval the number ```random()``` hits, we select the codon accordingly.

7. Write a function ```random_event``` that chooses a random event, given a probability distribution (set of events whose probabilities sum to 1).
You can use function ```random.uniform``` to produce values uniformly at random from the range $[0,1)$. The distribution should be given to your function as a dictionary from events to their probabilities.

In [9]:
import random

def random_event(dist):
    """
    Takes as input a dictionary from events to their probabilities.
    Return a random event sampled according to the given distribution.
    The probabilities must sum to 1.0
    """
    r = random.uniform(0, 1)

    dct = {}
    sum = 0
    for key, value in dist.items():
        sum += value
        dct[sum] = key

    for value, key in dct.items():
        if r <= value:
            return key
    return key


if __name__ == '__main__':
    distribution = dict(zip("ACGT", [0.10, 0.35, 0.15, 0.40]))
    print(", ".join(random_event(distribution) for _ in range(29)))

T, G, T, C, T, T, T, C, C, T, T, C, T, T, T, A, T, T, G, T, T, C, G, C, T, T, C, C, C


### Idea of solution

The idea of solution is to store the cumulative probabilities of the RNA based on the probability distribution given, and randomly choose one RNA by returning the first cumulative probability that is larger or equal than randomly chosen value (r) between 0 and 1. 

### Discussion

The program worked well apart from some values in future solutions where probabilities did not sum to 1.0, so second return statement was introduced as a way to return the RNA with cumulative maximum when the random r was higher than the sum of probabilities.

With this general routine, the following should be easy to solve.
 
8. Write a class ```ProteinToRandomRNA``` to produce a random RNA sequence encoding the input protein sequence according to the input codon adaptation probabilities. The actual conversion is done through the ```convert``` method. Run your program with `LTPIQNRA`.

In [10]:
class ProteinToRandomRNA(object):
    
    def __init__(self):
        self.aminos = get_dict_list()
        self.codon_prob = get_probabability_dict()

    def convert(self, s):
        protein = ""
        for c in s:
            distribution = {x:self.codon_prob[x] for x in self.aminos[c]}
            cod = random_event(distribution)
            protein = "".join((protein, cod))
        return protein
        
if __name__ == '__main__':
    protein_to_random_codons = ProteinToRandomRNA()
    print(protein_to_random_codons.convert("LTPIQNRA"))

CUGACCCCAAUACAGAAUCGUGCU


### Idea of solution

The idea of solution is similar to solution six, the difference is that the codon is chosen randomly by using the random_event function made in the previous solution, instead of the highest probability. 

### Discussion

The program works well and the results are as expected.

## Generating DNA sequences with higher-order Markov chains

We will now reuse the machinery derived above in a related context. We go back to DNA sequences, and consider some easy statistics that can be used to characterize the sequences. 
First, just the frequencies of bases $\texttt{A}$, $\texttt{C}$, $\texttt{G}$, $\texttt{T}$ may reveal the species from which the input DNA originates; each species has a different base composition that has been formed during evolution. 
More interestingly, the areas where DNA to RNA transcription takes place (coding region) have an excess of $\texttt{C}$ and $\texttt{G}$ over $\texttt{A}$ and $\texttt{T}$. To detect such areas a common routine is to just use a *sliding window* of fixed size, say $k$, and compute for each window position 
$T[i..i+k-1]$ the base frequencies, where $T[1..n]$ is the input DNA sequence. When sliding the window from  $T[i..i+k-1]$ to $T[i+1..i+k]$ frequency $f(T[i])$ gets decreases by one and $f(T[i+k])$ gets increased by one. 

9. Write a *generator* ```sliding_window``` to compute sliding window base frequencies so that each moving of the window takes constant time. We saw in the beginning of the course one way how to create generators using
  generator expression. Here we use a different way. For the function ```sliding_window``` to be a generator, it must have at least   one ```yield``` expression, see [https://docs.python.org/3/reference/expressions.html#yieldexpr](https://docs.python.org/3/reference/expressions.html#yieldexpr).
  
  Here is an example of a generator expression that works similarily to the built in `range` generator:
  ```Python
  def range(a, b=None, c=1):
      current = 0 if b == None else a
      end = a if b == None else b
      while current < end:
          yield current
          current += c
  ```
  A yield expression can be used to return a value and *temporarily* return from the function.

In [11]:
def sliding_window(s, k):
    """
    This function returns a generator that can be iterated over all
    starting position of a k-window in the sequence.
    For each starting position the generator returns the nucleotide frequencies
    in the window as a dictionary.
    """
    c = 0

    while c+k <= len(s):
        p = s[c:c+k]
        dct = {}
        for x in "ACGT":
            dct[x] = str.count(p, x)
        c += 1
        yield dct
        
    
if __name__ == '__main__':
    s = "TCCCGACGGCCTTGCC"
    for d in sliding_window(s, 4):
        print(d)

{'A': 0, 'C': 3, 'G': 0, 'T': 1}
{'A': 0, 'C': 3, 'G': 1, 'T': 0}
{'A': 1, 'C': 2, 'G': 1, 'T': 0}
{'A': 1, 'C': 2, 'G': 1, 'T': 0}
{'A': 1, 'C': 1, 'G': 2, 'T': 0}
{'A': 1, 'C': 1, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 1, 'T': 1}
{'A': 0, 'C': 2, 'G': 0, 'T': 2}
{'A': 0, 'C': 1, 'G': 1, 'T': 2}
{'A': 0, 'C': 1, 'G': 1, 'T': 2}
{'A': 0, 'C': 2, 'G': 1, 'T': 1}


### Idea of solution

The idea of solution is to split the string s to wanted size k starting from index c and then store the frequencies as a dictionary, which is yielded. 

### Discussion

The program works well and results are as expected.

 
Our models so far have been so-called *zero-order* models, as each event has been independent of other events. With sequences, the dependencies of events are naturally encoded by their *contexts*. Considering that a sequence is produced from left-to-right, a *first-order* context for $T[i]$ is $T[i-1]$, that is, the immediately preceding symbol. *First-order Markov chain* is a sequence produced by generating $c=T[i]$ with the probability of event of seeing symbol $c$ after previously generated symbol $a=T[i-1]$. The first symbol of the chain is sampled according to the zero-order model.  
The first-order model can naturally be extended to contexts of length $k$, with $T[i]$ depending on $T[i-k..i-1]$. Then the first $k$ symbols of the chain are sampled according to the zero-order model.  The following assignments develop the routines to work with the *higher-order Markov chains*. 
In what follows, a $k$-mer is a substring $T[i..i+k-1]$ of the sequence at an arbitrary position. 

10. Write function ```context_list``` that given an input DNA sequence $T$ associates to each $k$-mer $W$ the concatenation of all symbols $c$ that appear after context $W$ in $T$, that is, $T[i..i+k]=Wc$. For example, <span style="color:red; font:courier;">GA</span> is associated to <span style="color:blue; font: courier;">TCT</span> in $T$=<span style="font: courier;">AT<span style="color:red;">GA</span><span style="color:blue;">T</span>ATCATC<span style="color:red;">GA</span><span style="color:blue;">C</span><span style="color:red;">GA</span><span style="color:blue;">T</span>GTAG</span>, when $k=2$.

In [12]:
def context_list(s, k):
    dct = {}
    for x in range(k, len(s)):
        context = s[x-k:x]
        if context not in dct.keys():
            dct[context] = ""
        dct[context] += s[x]
    return dct
    
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATCTAG"
    d = context_list(s, k)
    print(d)

{'AT': 'GACCC', 'TG': 'A', 'GA': 'TCT', 'TA': 'TG', 'TC': 'AGT', 'CA': 'T', 'CG': 'AA', 'AC': 'G', 'CT': 'A'}


### Idea of solution

The idea of solution is to start going through the string s from index k, and store the k previous characters (context) as key to value at index k. Values are concatenated strings per key. 

### Discussion

The program works well and results are as expected.

11. With the above solution, write function ```context_probabilities``` to count the frequencies of symbols in each context and convert these frequencies into probabilities. Run `context_probabilities` with $T=$ `ATGATATCATCGACGATGTAG` and $k$ values 0 and 2.

In [13]:
def context_probabilities(s, k):
    dct = {}
    cl = context_list(s, k)
    for context, c in cl.items():
        window = list(sliding_window(c, len(c)))[0]
        probs = {}
        for x in "ACTG":
            if x not in window.keys():
                probs[x] = 0
            else:
                probs[x] = window[x] / len(c)
        dct[context] = probs
    return dct
    
if __name__ == '__main__':
    print(context_probabilities("ATGATATCATCGACGATGTAG", 2))

{'AT': {'A': 0.2, 'C': 0.4, 'T': 0.0, 'G': 0.4}, 'TG': {'A': 0.5, 'C': 0.0, 'T': 0.5, 'G': 0.0}, 'GA': {'A': 0.0, 'C': 0.3333333333333333, 'T': 0.6666666666666666, 'G': 0.0}, 'TA': {'A': 0.0, 'C': 0.0, 'T': 0.5, 'G': 0.5}, 'TC': {'A': 0.5, 'C': 0.0, 'T': 0.0, 'G': 0.5}, 'CA': {'A': 0.0, 'C': 0.0, 'T': 1.0, 'G': 0.0}, 'CG': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}, 'AC': {'A': 0.0, 'C': 0.0, 'T': 0.0, 'G': 1.0}, 'GT': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}}


### Idea of solution

The idea of solution is first to store the context_list as dictionary by using the function from previous solution and start going through the dictionary. Function sliding_window from solution nine is utilized to get the frequencies of the symbols for every context and made into a dictionary. Then every symbol is handled and added to a probability dictionary (probs). Lastly every context is stored into a dictionary as a key and its value is the probability dictionary probs. 

### Discussion

The program worked well and results are as expected.

12. With the above solution and the function ```random_event``` from the earlier exercise, write class ```MarkovChain```. Its ```generate``` method should generate a random DNA sequence following the original $k$-th order Markov chain probabilities. 

In [27]:
class MarkovChain:
    
    def __init__(self, zeroth, kth, k=2):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def generate(self, n, seed=None):
        if seed != None:
            random.seed(seed)
        chain = ""
        c = ""
        for i in range(n):
            if i < self.k or chain[i-self.k:i] not in self.kth:
                c = random_event(self.zeroth)
            else:
                c = random_event(self.kth[chain[i-self.k:i]])
            chain += c    
                
        return chain

if __name__ == '__main__':
    zeroth = {'A': 0.2, 'C': 0.19, 'T': 0.31, 'G': 0.3}
    kth = {'GT': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0},
           'CA': {'A': 0.0, 'C': 0.0, 'T': 1.0, 'G': 0.0},
           'TC': {'A': 0.5, 'C': 0.0, 'T': 0.0, 'G': 0.5},
           'GA': {'A': 0.0, 'C': 0.3333333333333333, 'T': 0.6666666666666666, 'G': 0.0},
           'TG': {'A': 0.5, 'C': 0.0, 'T': 0.5, 'G': 0.0},
           'AT': {'A': 0.2, 'C': 0.4, 'T': 0.0, 'G': 0.4},
           'TA': {'A': 0.0, 'C': 0.0, 'T': 0.5, 'G': 0.5},
           'AC': {'A': 0.0, 'C': 0.0, 'T': 0.0, 'G': 1.0},
           'CG': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}}
    n = 10    
    seed = 0
    mc = MarkovChain(zeroth, kth)
    print(mc.generate(n, seed))

GGTGGTGCTT


### Idea of solution

The idea of solution is concatenate to string chain randomly chosen symbol based on the given distributions. Zeroth distribution is used for the first k symbols and if no distribution is given for a sequence in the kth distribution. Kth distribution is used for all the rest. 

### Discussion

The program works well and results are as expected. 

If you have survived so far without problems, please run your program a few more times with different inputs. At some point you should get a lookup error in your hash-table! The reason for this is not your code, but the way we defined the model: Some $k$-mers may not be among the training data (input sequence $T$), but such can be generated as the first $k$-mer that is generated using the zero-order model.  

A general approach to fixing such issues with incomplete training data is to use *pseudo counts*. That is, all imaginable events are initialized to frequency count 1.   

13. Write a new solution `context_pseudo_probabilities` based on the solution to problem 11. But this time use pseudo counts in order to obtain a $k$-th order Markov chain that can assign a probability for any DNA sequence. You may use the standard library function `itertools.product` to iterate over all $k$-mer of given length (`product("ACGT", repeat=k)`).

In [15]:
def context_pseudo_probabilities(s, k):

    dct = {}
    cl = context_list(s, k)
    pseudo = ["".join(x) for x in product("ACGT", repeat=k)]
    for p in pseudo:
        if p not in cl.keys():
            cl[p] = ""
        cl[p] += "ACGT"
    for context, c in cl.items():
        window = list(sliding_window(c, len(c)))[0]
        probs = {}
        for x in "ACTG":
            if x not in window.keys():
                probs[x] = 0
            else:
                probs[x] = window[x] / len(c)
        dct[context] = probs
    
    return dct
    
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    kth = context_pseudo_probabilities(s, k)
    zeroth = context_pseudo_probabilities(s, 0)[""]
    print(f"zeroth: {zeroth}")
    print("\n".join(f"{k}: {dict(v)}" for k, v in kth.items()))
    
    print("\n", MarkovChain(zeroth, kth, k).generate(20))

zeroth: {'A': 0.32, 'C': 0.16, 'T': 0.28, 'G': 0.24}
AT: {'A': 0.2222222222222222, 'C': 0.3333333333333333, 'T': 0.1111111111111111, 'G': 0.3333333333333333}
TG: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'T': 0.3333333333333333, 'G': 0.16666666666666666}
GA: {'A': 0.14285714285714285, 'C': 0.2857142857142857, 'T': 0.42857142857142855, 'G': 0.14285714285714285}
TA: {'A': 0.16666666666666666, 'C': 0.16666666666666666, 'T': 0.3333333333333333, 'G': 0.3333333333333333}
TC: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'T': 0.16666666666666666, 'G': 0.3333333333333333}
CA: {'A': 0.2, 'C': 0.2, 'T': 0.4, 'G': 0.2}
CG: {'A': 0.5, 'C': 0.16666666666666666, 'T': 0.16666666666666666, 'G': 0.16666666666666666}
AC: {'A': 0.2, 'C': 0.2, 'T': 0.2, 'G': 0.4}
GT: {'A': 0.4, 'C': 0.2, 'T': 0.2, 'G': 0.2}
AA: {'A': 0.25, 'C': 0.25, 'T': 0.25, 'G': 0.25}
AG: {'A': 0.25, 'C': 0.25, 'T': 0.25, 'G': 0.25}
CC: {'A': 0.25, 'C': 0.25, 'T': 0.25, 'G': 0.25}
CT: {'A': 0.25, 'C': 0.25, 'T': 0.25, 

### Idea of solution

The idea of solution is similar to solution 11 with the exception of adding every possible combination of k length sequence as a key to the context dictionary and for every key concatenate string "ACGT" to the value, so the frequence for all the symbols of a context is at least one, when coutned in the sliding_window function.

### Discussion

The program works well and the results are as expected.

14. Write class ```MarkovProb``` that given the $k$-th order Markov chain developed above to the constructor, its method ```probability``` computes the probability of a given input DNA sequence.

In [16]:
class MarkovProb:
    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def probability(self, s):
        p = 1

        for i in range(len(s)):
            if i < self.k:
                p *= self.zeroth[s[i]]
            else:
                p *= self.kth[s[i-self.k:i]][s[i]]
        return p

    
if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovProb(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Probability of sequence {s} is {mc.probability(s)}")

Probability of sequence ATGATATCATCGACGATGTAG is 2.831270190340017e-10


### Idea of solution

The idea of solution is go through the string s, for index lower than k multiply p (starting from 1) with the probability of the symbol at the index from the zeroth probabilities, and for the rest multiply p with a probability from the kth probabilities based on the context (k length sequence) as key in the context dictionary, and the symbol at the index as key to the nested probability dictionary. 

### Discussion

The program works well and results are as expected.

With the last assignment you might end up in trouble with precision, as multiplying many small probabilities gives a really small number in the end. There is an easy fix by using so-called log-transform. 
Consider computation of $P=s_1 s_2 \cdots s_n$, where $0\leq s_i\leq 1$ for each $i$. Taking logarithm in base 2 from both sides gives $\log _2 P= \log _2 (s_1 s_2 \cdots s_n)=\log_2 s_1 + \log_2 s_2 + \cdots \log s_n= \sum_{i=1}^n \log s_i$, with repeated application of the property that the logarithm of a multiplication of two numbers is the sum of logarithms of the two numbers taken separately. The results is abbreviated as log-probability.

15. Write class ```MarkovLog``` that given the $k$-th order Markov chain developed above to the constructor, its method ```log_probability``` computes the log-probability of a given input DNA sequence. Run your program with $T=$ `ATGATATCATCGACGATGTAG` and $k=2$.

In [17]:
import math

class MarkovLog(object):

    def __init__(self, k, zeroth, kth):
        self.k = k
        self.log_zeroth = {}
        self.log_kth = {}

        for key, value in zeroth.items():
            self.log_zeroth[key] = math.log(value, 2)

        for item in kth.keys():
            dct = {}
            for key, value in kth[item].items():
                dct[key] = math.log(value, 2)
            self.log_kth[item] = dct
        
    def log_probability(self, s):
        p = 0
        for i in range(len(s)):
            if i < self.k:
                p += self.log_zeroth[s[i]]
            else:
                p += self.log_kth[s[i-self.k:i]][s[i]]
        return p
    
if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovLog(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Log probability of sequence {s} is {mc.log_probability(s)}")

Log probability of sequence ATGATATCATCGACGATGTAG is -31.717831515538315


### Idea of solution

The idea of solution is similar to previous solution, only the zeroth and kth probability dictionaries are modified to log values. 

### Discussion

The program works well and results are as expected.

Finally, if you try to use the code so far for very large inputs, you might observe that the concatenation of symbols following a context occupy considerable amount of space. This is unnecessary, as we only need the frequencies. 

16. Optimize the space requirement of your code from exercise 13 for the $k$-th order Markov chain by replacing the concatenations by direct computations of the frequencies. Implement this as the
  ```better_context_probabilities``` function.

In [18]:
def better_context_probabilities(s, k):
    dct = {}

    pseudo = ["".join(x) for x in product("ACGT", repeat=k)]
    for p in pseudo:
        dct[p] = {"A": 1, "C": 1, "G": 1, "T": 1}

    for i in range(k, len(s)):
        kth = s[i-k:i]
        cha = s[i]
        dct[kth][cha] += 1

    probs = {}
    
    for item in dct.keys():
        k_seq = dct[item]
        helper = {}
        sum = 0
        for value in k_seq.values():
            sum += value
        for key, value in k_seq.items():
           helper[key] = value / sum
        probs[item] = helper

    return probs

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    d = better_context_probabilities(s, k)
    print("\n".join(f"{k}: {v}" for k, v in d.items()))

AA: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AC: {'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2}
AG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AT: {'A': 0.2222222222222222, 'C': 0.3333333333333333, 'G': 0.3333333333333333, 'T': 0.1111111111111111}
CA: {'A': 0.2, 'C': 0.2, 'G': 0.2, 'T': 0.4}
CC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CG: {'A': 0.5, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.16666666666666666}
CT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GA: {'A': 0.14285714285714285, 'C': 0.2857142857142857, 'G': 0.14285714285714285, 'T': 0.42857142857142855}
GC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GT: {'A': 0.4, 'C': 0.2, 'G': 0.2, 'T': 0.2}
TA: {'A': 0.16666666666666666, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.3333333333333333}
TC: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.16666666666666666}
TG: {'A': 0.3333333333333333, 'C': 0.16666666666666

### Idea of solution

The idea of solution is first to create frequencies dictionary, with starting value of one for every symbol, for every possible sequence. Then go through the string s starting from k index and add one for the frequencies of the symbol (at index k) of the context (k length sequence before index k) frequencies. Lastly form a probability dictionary probs with helper dictionary, containing the probabilities of the symbols for context, as value to the context as a key. 

### Discussion

The program works well and results are as expected.

While the earlier approach of explicit concatenation of symbols following a context suffered from inefficient use of space, it does have a benefit of giving another much simpler strategy to sample from the distribution: 
observe that an element of the concatenation taken uniformly randomly is sampled exactly with the correct probability. 

17. Revisit the solution 12 and modify it to directly sample from the concatenation of symbols following a context. The function ```np.random.choice``` may be convenient here. Implement the modified version as the new `SimpleMarkovChain` class.

In [19]:
class SimpleMarkovChain(object):
    def __init__(self, s, k):
        self.s = s
        self.k = k
        
    def generate(self, n, seed=None):
        if seed != None:
            np.random.seed(seed)
        chain = ""
        dct = {}
        for x in range(self.k, len(self.s)):
            context = self.s[x-self.k:x]
            if context not in dct.keys():
                dct[context] = ""
            dct[context] += self.s[x]

        c = ""
        zeroth = np.array(["A", "C", "G", "T"])
        for i in range(n):
            if i < self.k or self.s[i-self.k:i] not in dct:        
                c = np.random.choice(zeroth)
            else:
                arr = np.array(list(dct[self.s[i-self.k:i]]))
                c = np.random.choice(arr)
            chain += c    

                
        return chain
        
        
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    n = 10
    seed = 7
    mc = SimpleMarkovChain(s, k)
    print(mc.generate(n, seed))

TAAATATGGT


### Idea of solution

The idea of solution is first to form a context dictionary similarly to solution 10, then the zeroth symbol-array is created and lastly, similar to solution 12, the random symbol is chosen, but now using numpy.random.choice. 

### Discussion

The program works well and results are as expected. The code is a little hard-to-read in some places.

## $k$-mer index

Our $k$-th order Markov chain can now be modified to a handy index structure called $k$-mer index. This index structure associates to each $k$-mer its list of occurrence positions in DNA sequence $T$.  Given a query $k$-mer $W$, one can thus easily list all positions $i$ with  $T[i..k-1]=W$.

18. Implement function ```kmer_index``` inspired by your earlier code for the $k$-th order Markov chain. Test your program with `ATGATATCATCGACGATGTAG` and $k=2$.

In [29]:
def kmer_index(s, k):
    dct = {}
    for x in range(k, len(s) + 1):
        context = s[x-k:x]
        if context not in dct.keys():
            dct[context] = []
        dct[context].append(x - k)
    return dct

if __name__ == '__main__':
    k=2
    s = "ATGATATCATCGACGATGTAG"
    print("Using string:")
    print(s)
    print("".join([str(i%10) for i in range(len(s))]))
    print(f"\n{k}-mer index is:")
    d=kmer_index(s, k)
    print(dict(d))

Using string:
ATGATATCATCGACGATGTAG
012345678901234567890

2-mer index is:
{'AT': [0, 3, 5, 8, 15], 'TG': [1, 16], 'GA': [2, 11, 14], 'TA': [4, 18], 'TC': [6, 9], 'CA': [7], 'CG': [10, 13], 'AC': [12], 'GT': [17], 'AG': [19]}


### Idea of solution

The idea of solution is similar to solution 10, but instead of the concatenated string of symbols, a list of indexes of the context's location in the string s is stored to the corresponding context.

### Discussion

The program works well and results are as expected.

## Comparison of probability distributions

Now that we know how to learn probability distributions from data, we might want to compare two such distributions, for example, to test if our programs work as intended. 

Let $P=\{p_1,p_2,\ldots, p_n\}$ and $Q=\{q_1,q_2,\ldots, q_n\}$ be two probability distributions for the same set of $n$ events. This means $\sum_{i=1}^n p_i=\sum_{i=1}^n q_i=1$, $0\leq p_j \leq 1$, and $0\leq q_j \leq 1$ for each event $j$. 

*Kullback-Leibler divergence* is a measure $d()$ for the *relative entropy* of $P$ with respect to $Q$ defined as 
$d(P||Q)=\sum_{i=1}^n p_i \log\frac{p_i}{q_i}$.


This measure is always non-negative, and 0 only when $P=Q$. It can be interpreted as the gain of knowing $Q$ to encode $P$. Note that this measure is not symmetric.

19. Write function ```kullback_leibler``` to compute $d(P||Q)$. Test your solution by generating a random RNA sequence
  encoding the input protein sequence according to the input codon adaptation probabilities.
  Then you should learn the codon adaptation probabilities from the RNA sequence you generated.
  Then try the same with uniformly random RNA sequences (which don't have to encode any
  specific protein sequence). Compute the relative entropies between the
  three distribution (original, predicted, uniform) and you should observe a clear difference.
  Because $d(P||Q)$ is not symmetric, you can either print both $d(P||Q)$ and $d(Q||P)$,
  or their average.
  
  This problem may be fairly tricky. Only the `kullback_leibler` function is automatically tested. The codon probabilities is probably a useful helper function. The main guarded section can be completed by filling out the `pass` sections using tooling from previous parts and fixing the *placeholder* lines.

In [30]:
def codon_probabilities(rna):
    """
    Given an RNA sequence, simply calculates the proability of
    all 3-mers empirically based on the sequence
    """
    dct = {"".join(codon): 0 for codon in product("ACGU", repeat=3)}

    for i in range (3, len(rna), 3):
        dct[rna[i-3:i]] += 1

    adapt = {}
    for key, value in dct.items():
        amino = rna_to_prot(key)
        if amino not in adapt:
            adapt[amino] = {}
            adapt[amino]["sum"] = 0
        adapt[amino][key] = value
        adapt[amino]["sum"] += value

    probs = {}
    for codon_dct in adapt.values():
        for codon, freq in codon_dct.items():
            if codon == "sum":
                continue
            probs[codon] = freq / codon_dct["sum"]

    return probs
    
def kullback_leibler(p, q):
    """
    Computes Kullback-Leibler divergence between two distributions.
    Both p and q must be dictionaries from events to probabilities.
    The divergence is defined only when q[event] == 0 implies p[event] == 0.
    """

    kl = 0
    for cod in p.keys():
        pcod = p[cod]
        qcod = q[cod]
        if pcod == 0:
            continue
        kl += pcod * math.log2(pcod/qcod)

    return kl

if __name__ == '__main__':
    aas = list("*ACDEFGHIKLMNPQRSTVWY") # List of amino acids
    n = 10000
    
    # generate a random protein and some associated rna
    protein = "".join(choice(aas, n)) 
    protein_to_rna = ProteinToRandomRNA()
    rna = protein_to_rna.convert(protein)
    
    # Maybe check that converting back to protein results in the same sequence
    back_convert = rna_to_prot(rna)
    #print("Converted right: ", protein == back_convert)
    
    # Calculate codon probabilities of the rna sequence
    cp_predicted = codon_probabilities(rna) # placeholder call
    
    # Calculate codon probabilities based on the codon usage table
    cp_orig = get_probabability_dict() # placeholder dict
    
    
    # Create a completely random RNA sequence and get the codon probabilities
    lst = ["".join(codon) for codon in product("AGCU", repeat=3)]
    uni = "".join(choice(lst, n))
    cp_uniform = codon_probabilities(uni) # placeholder call
    
    print("d(original || predicted) =", kullback_leibler(cp_orig, cp_predicted))
    print("d(predicted || original) =", kullback_leibler(cp_predicted, cp_orig))
    print()
    print("d(original || uniform) =", kullback_leibler(cp_orig, cp_uniform))
    print("d(uniform || original) =", kullback_leibler(cp_uniform, cp_orig))
    print()
    print("d(predicted || uniform) =", kullback_leibler(cp_predicted, cp_uniform))
    print("d(uniform || predicted) =", kullback_leibler(cp_uniform, cp_predicted))

d(original || predicted) = 0.08855637473570943
d(predicted || original) = 0.030552823762114767

d(original || uniform) = 1.4442435681802144
d(uniform || original) = 1.5039624871549973

d(predicted || uniform) = 1.5580090636750372
d(uniform || predicted) = 1.6907388604520432


### Idea of solution

The idea of solution is first to generate a random protein and convert it to RNA. This RNA is then used to produce a prediction of probabilities for a particular amino acid to be encoded by a 3-mer. This is achieved in the function codon_probabilities.

In the function a dictionary dct is generated holding every possible 3-mer combination with initial value of 0. The RNA sequence is then iterated and the frequencies of the 3-mer are stored. Then a second dictionary adapt is generated, holding amino acid encoded by a 3-mer as key a nested dictionary as value: the nested dictionary holds the sum of frequencies of 3-mers for a amino acid and the key-value pair from dct dictionary. Lastly the probability dictionary probs is generated, counting the probabilities for a 3-mer by dividing its frequence with the sum of frequencies of 3-mers for a amino acid.  

After the predicted probabilities, the original probabilities are stored into a dictionary using a function introduced in solution five. The last probability dictionary, with uniform probabilities for a 3-mer, is generated with np.random.choice and codon_probabilities.

These probabilities are then compared with kullback_leibler function which simply computes the Kullback-Leibler divergence between two distributions.

### Discussion

The program works well and results are as expected.